In [1]:
!nvidia-smi

Tue Feb 16 09:51:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/gdrive/MyDrive/kaggle_datasets'

In [9]:
%cd '/content/gdrive/MyDrive/kaggle_datasets'

/content/gdrive/MyDrive/kaggle_datasets


In [10]:
!ls

checkpoint					   kaggle.json
colab_fruits.h5					   messy-vs-clean-room
colab_fruits_weights.index			   model-for-detecting-malaria
colab_inception_resnet_malaria_detection_model.h5  natural-images
indian-food-classification			   real-and-fake-face-detection
intel-image-classification


In [11]:
%cd intel-image-classification

/content/gdrive/MyDrive/kaggle_datasets/intel-image-classification


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_dir = '/content/gdrive/MyDrive/kaggle_datasets/intel-image-classification/seg_train/seg_train'
val_dir = '/content/gdrive/MyDrive/kaggle_datasets/intel-image-classification/seg_test/seg_test'

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150,150),
                                                    batch_size=16,
                                                    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(val_dir,
                                                        target_size=(150,150),
                                                        batch_size=16,
                                                        class_mode='categorical')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [5]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [12]:
model = models.Sequential([
                           layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(150,150,3)),
                           layers.BatchNormalization(),
                           layers.MaxPooling2D((2,2)),
                           layers.Dropout((0.4)),


                           layers.Conv2D(32, (3,3), activation='relu', padding='same'),
                           layers.Conv2D(64, (3,3), activation='relu', padding='same'),
                           layers.BatchNormalization(),
                           layers.MaxPooling2D((2,2)),
                           layers.Dropout((0.4)),

                           layers.Conv2D(64, (3,3), activation='relu', padding='same'),
                           layers.Conv2D(128, (3,3), activation='relu', padding='same'),
                           layers.BatchNormalization(),
                           layers.MaxPooling2D((2,2)),
                           layers.Dropout((0.4)),

                           layers.Flatten(),

                           layers.Dense(256, activation='relu'),
                           layers.BatchNormalization(),
                           layers.Dense(512, activation='relu'),
                           layers.Dense(6, activation='softmax')

])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 150, 150, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 64)        2

In [14]:
model.compile(tf.keras.optimizers.Adam(learning_rate=2e-4),
              tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

In [15]:
history = model.fit_generator(train_generator,
                             steps_per_epoch=878,
                             epochs=10,
                             validation_data=validation_generator,
                            validation_steps=188)

model.save('colab_intel_image_classification_custom.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
878/878 [==============================] - 5730s 7s/step - loss: 1.2553 - accuracy: 0.5097 - val_loss: 4.7962 - val_accuracy: 0.4323
Epoch 2/10
878/878 [==============================] - 104s 118ms/step - loss: 0.9556 - accuracy: 0.6363 - val_loss: 5.8011 - val_accuracy: 0.3500
Epoch 3/10
878/878 [==============================] - 104s 118ms/step - loss: 0.8821 - accuracy: 0.6608 - val_loss: 4.0798 - val_accuracy: 0.4360
Epoch 4/10
878/878 [==============================] - 103s 117ms/step - loss: 0.8470 - accuracy: 0.6803 - val_loss: 3.1789 - val_accuracy: 0.4260
Epoch 5/10
878/878 [==============================] - 103s 117ms/step - loss: 0.7917 - accuracy: 0.7084 - val_loss: 1.8521 - val_accuracy: 0.6113
Epoch 6/10
878/878 [==============================] - 103s 118ms/step - loss: 0.7583 - accuracy: 0.7225 - val_loss: 0.8946 - val_accuracy: 0.7003
Epoch 7/10
878/878 [==============================] - 103s 117ms/step - loss: 0.7171 - accuracy: 0.7363 - val_loss: 0.9366 - v

PREDICTIONS :

In [74]:
def predictions(img_path):
    img = image.load_img(img_path, target_size=(150, 150, 3)) # Loads the image at size (224, 224, 3)
    
    img_array = image.img_to_array(img)   # Converts the jpg file into numpy array
    
    img_batch = np.expand_dims(img_array, axis=0)  # Expanding the dimensions of the array
    
    img_batch = img_batch/255.  # Scaling down the value to bring it in range[0, 1]
    
    pred = model.predict(img_batch)[0]  # Will get a value in range[0, 1]
    
    print(pred)   

In [77]:
def img_preprocess(img_path):
  img = image.load_img(img_path, target_size=(150, 150, 3)) # Loads the image at size (224, 224, 3)
    
  img_array = image.img_to_array(img)   # Converts the jpg file into numpy array
    
  img_batch = np.expand_dims(img_array, axis=0)  # Expanding the dimensions of the array
    
  final_img = img_batch / 255.  # Scaling down the value to bring it in range[0, 1]

  return final_img
    

In [18]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image

In [16]:
img1 = '/content/gdrive/MyDrive/kaggle_datasets/intel-image-classification/seg_pred/seg_pred/10004.jpg'

In [ ]:
img1 = image.load_img(img1, target_size=(150, 150, 3))
plt.imshow(img1)
plt.show()

In [12]:
model = models.load_model('colab_intel_image_classification_custom.h5')

In [24]:
a = predictions(img1)

[[0.01704202 0.94110286 0.00812446 0.0169106  0.00525763 0.0115624 ]]


In [25]:
np.argmax(a)

0

In [26]:
img2 = '/content/gdrive/MyDrive/kaggle_datasets/intel-image-classification/seg_pred/seg_pred/10005.jpg'

In [28]:
b = predictions(img2)

[[1.9070917e-03 1.3631044e-02 1.4417425e-01 6.7576110e-01 1.6433720e-01
  1.8930402e-04]]


In [29]:
np.argmax(b)

0

In [30]:
img3 = '/content/gdrive/MyDrive/kaggle_datasets/intel-image-classification/seg_pred/seg_pred/10012.jpg'

In [53]:
c = predictions(img3)

[0.0143818  0.9196803  0.02119784 0.00127536 0.0162883  0.02717639]


In [55]:
np.argmax(c)

0

In [ ]:
list(c)

In [64]:
lst = [0.0143818,  0.9196803,  0.02119784, 0.00127536, 0.0162883,  0.02717639]
lst

[0.0143818, 0.9196803, 0.02119784, 0.00127536, 0.0162883, 0.02717639]

In [60]:
classes = {0:'buildings', 1: 'forest', 2 : 'glacier', 3 : 'mountain',
           4 : 'sea', 5 : 'street'}

In [61]:
classes[0]

'buildings'

In [65]:
np.argmax(lst)

1

In [66]:
a = classes[np.argmax(lst)]
a

'forest'

In [69]:
predictions(img2)

[1.9070917e-03 1.3631044e-02 1.4417425e-01 6.7576110e-01 1.6433720e-01
 1.8930402e-04]


In [90]:
one = img_preprocess(img2)

In [91]:
pr1 = model.predict(one)
pr1

array([[1.9070917e-03, 1.3631044e-02, 1.4417425e-01, 6.7576110e-01,
        1.6433720e-01, 1.8930402e-04]], dtype=float32)

In [92]:
np.argmax(pr1)

3

In [93]:
pred_names = classes[np.argmax(pr1)]
pred_names

'mountain'

Final Predictions :

1. img_path
2. pre process
3. model.predict
4. pred_names = classes[np.argmax(pr1)]
  pred_names

In [101]:
def prediction_names(img_path):
  img = image.load_img(img_path, target_size=(150, 150, 3)) # Loads the image at size (224, 224, 3)
    
  img_array = image.img_to_array(img)   # Converts the jpg file into numpy array
    
  img_batch = np.expand_dims(img_array, axis=0)  # Expanding the dimensions of the array
    
  final_img = img_batch / 255.  # Scaling down the value to bring it in range[0, 1]

  pred = model.predict(final_img)

  classes = {0:'buildings', 1: 'forest', 2 : 'glacier', 3 : 'mountain',
           4 : 'sea', 5 : 'street'}

  pred_names = classes[np.argmax(pred)]

  return pred_names

In [102]:
img4 = '/content/gdrive/MyDrive/kaggle_datasets/intel-image-classification/seg_pred/seg_pred/10038.jpg'

In [103]:
prediction_names(img4)

'sea'

In [94]:
img5 = '/content/gdrive/MyDrive/kaggle_datasets/intel-image-classification/seg_pred/seg_pred/10017.jpg'